# 1. Pisahkan antara variabel yang dapat digunakan dan variabel yang tidak dapat digunakan.

In [1]:
import pandas as pd
import numpy as np

PATH = "/content/wbc.csv"

df = pd.read_csv(PATH)

df.dropna(axis=1, how='all', inplace=True)

print("Ukuran dataset:", df.shape)
display(df.head())
print("\nInfo kolom:")
print(df.info())
print("\nUnique values per kolom (contoh):")
print(df.nunique().sort_values(ascending=False).head(20))

Ukuran dataset: (569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678



Info kolom:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se 

# 2. Lakukan proses encoding pada kolom "diagnosis".

In [2]:
unusable = [c for c in df.columns if c.lower() == 'id' or c.lower().startswith('unnamed')]

if 'diagnosis' not in df.columns:
    raise ValueError("Kolom 'diagnosis' tidak ditemukan.")

usable_features = [c for c in df.columns if c not in unusable + ['diagnosis']]

print("Unusable columns:", unusable)
print("Jumlah fitur usable:", len(usable_features))
print(usable_features)

Unusable columns: ['id']
Jumlah fitur usable: 30
['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']


In [5]:
df['diagnosis'] = df['diagnosis'].astype(str).str.strip()
mapping = {'M': 1, 'B': 0}

df['diagnosis_encoded'] = df['diagnosis'].map(mapping)

if df['diagnosis_encoded'].isna().any():
    print("Ada nilai diagnosis yang tidak M atau B:")
    print(df.loc[df['diagnosis_encoded'].isna(), 'diagnosis'].unique())
else:
    print("Semua diagnosis ter-encode dengan baik.")

y = df['diagnosis_encoded']

Semua diagnosis ter-encode dengan baik.


#3. Lakukan proses standarisasi pada semua kolom yang memiliki nilai numerik.



In [7]:
from sklearn.preprocessing import StandardScaler

numeric_features = [c for c in usable_features if pd.api.types.is_numeric_dtype(df[c])]
non_numeric_features = [c for c in usable_features if c not in numeric_features]

print("Numeric features (yang akan distandarisasi):", len(numeric_features))
if non_numeric_features:
    print("WARNING: fitur non-numeric dan di-ignore:", non_numeric_features)

X_numeric = df[numeric_features].copy()

scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(X_numeric)
X_scaled = pd.DataFrame(X_scaled_arr, columns=numeric_features, index=df.index)

df_prepared = pd.concat([X_scaled, y.rename('diagnosis')], axis=1)
display(df_prepared.head())

Numeric features (yang akan distandarisasi): 30


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015,1
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190,1
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391,1
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010,1
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100,1


#4. Lakukan proses stratified split data untuk membuat data latih dan data uji dengan rasio 80:20.

In [8]:
from sklearn.model_selection import train_test_split

X = X_scaled
y = y

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("Shapes:")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train distribusi:")
print(y_train.value_counts(normalize=False))
print(y_test.value_counts(normalize=False))
print("\nProporsi kelas (train):")
print(y_train.value_counts(normalize=True))
print("Proporsi kelas (test):")
print(y_test.value_counts(normalize=True))


Shapes:
X_train: (455, 30)
X_test : (114, 30)
y_train distribusi:
diagnosis_encoded
0    285
1    170
Name: count, dtype: int64
diagnosis_encoded
0    72
1    42
Name: count, dtype: int64

Proporsi kelas (train):
diagnosis_encoded
0    0.626374
1    0.373626
Name: proportion, dtype: float64
Proporsi kelas (test):
diagnosis_encoded
0    0.631579
1    0.368421
Name: proportion, dtype: float64
